# 텐서플로 허브를 사용한 영화 리뷰 텍스트 분류하기

본 튜토리얼은 텐서플로우 [공식 사이트 튜토리얼](https://www.tensorflow.org/tutorials/keras/text_classification_with_hub)을 참고하여 작성되었습니다.

이번 튜토리얼에서는 텐서플로우 허브를 사용한 기초적인 전이 학습(transfer learning) 애플리케이션을 학습합니다.

앞서 영화 리뷰 텍스트 분류하기와 마찬가지로 인터넷 영화 데이터베이스(Internet Movie Database)에서 수집한 50,000개의 영화 리뷰 텍스트를 담은 IMDB 데이터셋을 사용합니다. 25,000개는 훈련 세트, 25,000개는 테스트 세트로 나뉘어져 있고, 훈련 세트와 테스트 세트의 긍정적인 리뷰와 부정적인 리뷰의 개수는 서로 동일합니다.

앞선 튜토리얼과 마찬가지로 영화 리뷰 텍스트를 긍정(1) 또는 부정(0)으로 **이진 분류**하는 모델을 학습시키려고 합니다.

모델을 구성하고 훈련하기 위해 케라스와 더불어 전이 학습 라이브러리이자 플랫폼인 [텐서플로 허브](https://www.tensorflow.org/hub)를 사용합니다.

In [1]:
import numpy as np
import tensorflow as tf

!pip install tensorflow-hub
!pip install tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("버전: ", tf.__version__)
print("즉시 실행 모드: ", tf.executing_eagerly())
print("허브 버전: ", hub.__version__)
print("GPU", "사용 가능" if tf.config.experimental.list_physical_devices("GPU") else "사용 불가능")

     |████████████████████████████████| 3.9MB 9.0MB/s 
버전:  2.5.0
즉시 실행 모드:  True
허브 버전:  0.12.0
GPU 사용 가능


## 1. IMDB 데이터셋 다운로드

IMDB 데이터셋은 Tensorflow datasets 또는 imdb reviews에 포함되어 있습니다.(앞선 튜토리얼에서는 imdb reviews에 포함된 데이터셋을 다운로드 해보았습니다) IMDB 데이터셋을 Tensorflow datasets으로부터 컴퓨터(또는 코랩 런타임)에 다운로드합니다.

In [2]:
# 훈련 세트를 6(훈련 샘플) : 4(검증 샘플)로 나눕니다.
# train_data에 샘플과 레이블이 함께 저장됩니다.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## 2. 데이터 탐색

데이터셋의 샘플은 전처리된 정수 배열입니다. 각 정수들은 리뷰에 나오는 단어를 나타냅니다. 레이블은 0 또는 1입니다.

처음 10개의 샘플을 출력해 보겠습니다.

In [3]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

처음 10개의 레이블도 출력해 보겠습니다.

In [4]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

## 3. 모델 구성

layer를 쌓기 위해 3가지를 고려해야 합니다.

+ 텍스트를 어떻게 표현할 것인가?
+ 얼마나 많은 층을 사용할 것인가?
+ 각 층에서 얼마나 많은 Hidden Unit을 사용할 것인가?

여러 가지 텍스트를 표현하는 방법 중에 한 가지 방법은 문장을 임베딩 벡터로 바꾸는 것입니다. 그러면 첫 번째 Layer로 사전 훈련(pre-trained)된 텍스트 임베딩을 사용할 수 있는데, 아래와 같은 장점이 있습니다.

+ 텍스트 전처리에 대해 신경 쓸 필요가 없습니다.(패딩을 신경 쓸 필요가 없다)
+ 전이 학습의 장점을 이용할 수 있습니다.
+ 임베딩은 고정 크기이기 때문에 처리 과정이 단순해집니다.

이번 예제는 [텐서플로 허브](https://www.tensorflow.org/hub)에 있는 **사전 훈련된 텍스트 임베딩 모델**인 [google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1)을 사용하겠습니다.

테스트해 볼 수 있는 사전 훈련된 모델이 3가지 더 있습니다.

* [google/tf2-preview/gnews-swivel-20dim-with-oov/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1) : [google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1)와 동일하지만 어휘 사전(vocabulary)의 2.5%가 OOV 버킷(bucket)으로 변환되었습니다. 해당 문제의 어휘 사전과 모델의 어휘 사전이 완전히 겹치지 않을 때 도움이 됩니다.
* [google/tf2-preview/nnlm-en-dim50/1](https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1) : 더 큰 모델입니다. 차원 크기는 50이고 어휘 사전의 크기는 1백만 개 이하입니다.
* [google/tf2-preview/nnlm-en-dim128/1](https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1) : 훨씬 더 큰 모델입니다. 차원 크기는 128이고 어휘 사전의 크기는 1백만 개 이하입니다.

먼저 문장을 임베딩시키기 위해 텐서플로 허브 모델을 사용하는 `keras`층을 만들어 보겠습니다. 그 다음으로 몇 개의 샘플을 입력하여 테스트 해보겠습니다. 입력 텍스트의 길이에 상관없이 임베딩 출력 크기는 `(num_examples, embedding_dimension)`가 됩니다. (앞선 튜토리얼에서 모든 샘플들의 텍스트 길이를 똑같이 맞춰주기 위해 패딩(padding)을 진행했지만, 이를 위한 과정이 필요 없어졌다.)

In [5]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

이제 전체 모델을 만들어 보겠습니다.

In [6]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


## 4. 모델 컴파일

손실함수는 이진 분류 문제에 모델이 확률을 출력하므로 `binary_crossentropy` 손실 함수를 사용하겠습니다.

옵티마이저로는 Adam 알고리즘을 구현한 AdamOptimizer를 사용합니다.

In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## 5. 모델 훈련

512개의 샘플로 이루어진 미니 배치(mini-batch)로 나누고 훈련 데이터셋 15,000개 중 무작위로 10,000개를 선별해 20번 반복 훈련합니다. 검증 데이터셋 10,000개로 모델의 손실과 정확도를 모니터링합니다.

In [8]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 6s 78ms/step - loss: 0.7448 - accuracy: 0.5226 - val_loss: 0.6803 - val_accuracy: 0.5619
Epoch 2/20
30/30 [==============================] - 2s 57ms/step - loss: 0.6412 - accuracy: 0.5933 - val_loss: 0.6078 - val_accuracy: 0.6370
Epoch 3/20
30/30 [==============================] - 2s 56ms/step - loss: 0.5808 - accuracy: 0.6690 - val_loss: 0.5604 - val_accuracy: 0.6915
Epoch 4/20
30/30 [==============================] - 2s 56ms/step - loss: 0.5309 - accuracy: 0.7164 - val_loss: 0.5168 - val_accuracy: 0.7272
Epoch 5/20
30/30 [==============================] - 2s 56ms/step - loss: 0.4831 - accuracy: 0.7563 - val_loss: 0.4779 - val_accuracy: 0.7655
Epoch 6/20
30/30 [==============================] - 2s 55ms/step - loss: 0.4392 - accuracy: 0.7902 - val_loss: 0.4443 - val_accuracy: 0.7757
Epoch 7/20
30/30 [==============================] - 2s 55ms/step - loss: 0.4015 - accuracy: 0.8139 - val_loss: 0.4149 - val_accuracy: 0.7975
Epoch 8/20
30

## 6. 모델 평가

모델의 성능을 확인해봅시다. 손실과 정확도가 반환됩니다.

In [9]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.3239 - accuracy: 0.8524
loss: 0.324
accuracy: 0.852
